In [3]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import random
import json
import os
import shutil
from pathlib import Path
import os.path
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
def coco_to_yolo_bb(x1, y1, w, h, image_w, image_h):
    #"bbox": [x,y,width,height], (COCO)
    #x_center y_center width height (YOLO)
    return [((2*x1 + w)/(2*image_w)) , ((2*y1 + h)/(2*image_h)), w/image_w, h/image_h]

In [4]:
#dataDir='/Users/azakaria/Downloads'
#dataType='val2017'
annFile=f'/Users/azakaria/Documents/yolo datasets/nightowls/nightowls_validation.json'



In [5]:
# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: '/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_train1/nightowls_validation'

In [4]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
cat_map = {}
for i,cat in enumerate(cats):
    cat_map[cat['id']] = i
nms=[cat['name'] for cat in cats]

In [5]:
catIds = coco.getCatIds()
def get_imgs_from_each_category(catIds, numImgs=None):
    imgIds = []
    for catId in catIds:
        imgIds_current_batch = coco.getImgIds(catIds=[catId])[:numImgs]
        if(len(imgIds_current_batch) < 30):
            print('ERROR! Not 30 images in this category!')
        imgIds = imgIds + imgIds_current_batch
    return imgIds
cat_map

{1: 0, 2: 1, 3: 2, 4: 3}

In [12]:
"""
getImgIds() -> list[imgId] 
loadImgs(ids : [imgId]) -> list[img]
getAnnIds(imgIds: list[imgId]) -> list[annId]
loadAnns(ids: list[annId]) -> list[ann]

*One imgId can be multiple annIds
*One annId is one ann


#imgIds need to have annotations and those annotations bboxes must not be negative
#write_label_files need
    #filename, which are in imgs
    #catIds, which are in anns
    #bboxes, which are in anns
"""
def get_random_img_ids(numImgs):
    imgIds = coco.getImgIds()
    random.shuffle(imgIds)
    return imgIds[:numImgs]

def get_img_ids_per_cat(numImgs_per_cat):
    catIds = coco.getCatIds()
    imgIds_total = []
    for catId in catIds:
        imgIds_cat = coco.getImgIds(catIds=[catId])
        #imgIds = coco.getImgIds()
        random.shuffle(imgIds_cat)
        imgIds_current = []
        for imgId in imgIds_cat:
            if imgId_has_annotation(imgId) and imgId_has_positive_bboxes(imgId):
                imgIds_current.append(imgId)
            if(len(imgIds_current) == numImgs_per_cat):
                break
        imgIds_total = imgIds_total + imgIds_current
    return imgIds_total

#imgIds = get_img_ids(120)

def imgId_has_positive_bboxes(imgId):
    annIds = coco.getAnnIds(imgIds=imgId)
    anns = coco.loadAnns(annIds)
    for ann in anns:
        negative_bbox = False 
        for bbox_val in ann['bbox']:
            if bbox_val < 0:
                negative_bbox = True
        if negative_bbox == True:
            return False
    return True

def imgId_has_annotation(imgId):
    annIds = coco.getAnnIds(imgIds=imgId)
    if(len(annIds) > 0):
        return True
    else:
        return False

#def write_label_files(numImgs_per_cat):
def write_label_files(numImgs):
    """
    Some images:
    *Do not have annotations.
    *Have negative bboxes
    """
    #imgIds = get_img_ids_per_cat(numImgs_per_cat)
    imgIds = get_random_img_ids(numImgs)

    for imgId in imgIds:
        img = coco.loadImgs(imgId)[0] #If only 1 imgId, len() = 1
        label_dir = Path('/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_train1/dataset/labels')
                        #'/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_val1/dataset/labels'
        filename = img['file_name'].split('.')[0] + '.txt'
        with open(label_dir/filename, 'w') as file:
            annIds = coco.getAnnIds(imgIds = imgId, catIds=catIds)
            anns = coco.loadAnns(annIds)
            #print(len(anns))
            #print('-----------------------------')
            #anns = remove_negative_bbox_anns(anns)
            for i,ann in enumerate(anns):
                x_center, y_center, width, height = coco_to_yolo_bb(*ann['bbox'], img['width'], img['height'])
                print(x_center, y_center, width, height)
                file.write(f'{cat_map[ann["category_id"]]} {x_center} {y_center} {width} {height}\n')
    return

def copy_images_to_dataset_dir(src_image_dir, dest_image_dir, image_extension):
    """
    x number of labels are written to the labels dir by write_label_files(x)
    This function copies the corresponding images into the sibling images dir, completing the dataset folder for YOLO training
    """
    fs = os.listdir('/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_train1/labels')
    for f in fs:
        src = str(src_image_dir/f.split('.')[0]) + image_extension
        dest = str(dest_image_dir/ f.split('.')[0]) + image_extension
        if os.path.isfile(src):
            shutil.copyfile(src, dest)
        else:
            print(f'{src} is not a file')
    return

#Currently outputting 473 instead of 480 (4*120)
#So one of more categories has fewer than 120 valid images, it *seems*
if __name__ == "__main__":
    #os.system('rm -rf /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images/*; rm -rf /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels/*')
    #write_label_files(120) #dataset/labels
    write_label_files(480) #dataset/labels
    copy_images_to_dataset_dir(Path('/Users/azakaria/Documents/yolo datasets/nightowls/nightowls_validation'),Path('/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_val1/dataset/images/'), '.png') #dataset/images
    #os.system('/usr/local/bin/docker exec -ti 5dcf6e04dbba rm -rf /usr/src/datasets/nightowls_val/images')
    #os.system('/usr/local/bin/docker exec -ti 5dcf6e04dbba rm -rf /usr/src/datasets/nightowls_val/labels')
    #os.system('/usr/local/bin/docker cp /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/. 5dcf6e04dbba:/usr/src/datasets/nightowls_val')
    #print('ls /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images | wc -l ; ls /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels | wc -l')
    #os.system('ls /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images | wc -l ; ls /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels | wc -l')

#rm -rf /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images/*; rm -rf /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels/*
#docker cp /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/. 5dcf6e04dbba:/usr/src/datasets/nightowls_val
#ls /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images | wc -l ; ls /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels | wc -l


#region useful test code for single imgId
#imgId = 7000058
""""
imgIds = coco.getImgIds()
imgId = imgIds[2]
img = coco.loadImgs(ids = imgId)[0]
annIds = coco.getAnnIds(imgIds = imgId)
anns = coco.loadAnns(annIds)
print(coco)

fp = f'/Users/azakaria/Downloads/nightowls_validation/{img["file_name"]}' 
I = io.imread(fp)
plt.axis('off')
plt.imshow(I)
plt.show()

plt.imshow(I); plt.axis('off')
coco.showAnns(anns, draw_bbox=True)
"""
#endregion

0.54931640625 0.415625 0.0283203125 0.115625
0.57177734375 0.4234375 0.0244140625 0.10625
0.8720703125 0.5234375 0.177734375 0.171875
0.396484375 0.40390625 0.017578125 0.0796875
-0.04150390625 0.5015625 0.0732421875 0.4375
0.5615234375 0.3515625 0.021484375 0.04375
0.44384765625 0.38046875 0.0302734375 0.1046875
0.2265625 0.41171875 0.037109375 0.1390625
0.2041015625 0.39765625 0.029296875 0.1328125
0.6708984375 0.4453125 0.041015625 0.18125
0.94970703125 0.43515625 0.0439453125 0.2203125
0.76025390625 0.39296875 0.0517578125 0.2578125
0.65771484375 0.4125 0.0595703125 0.284375
0.7734375 0.48671875 0.02734375 0.1328125
0.72314453125 0.484375 0.0244140625 0.09375
0.74853515625 0.48125 0.0205078125 0.096875
0.63427734375 0.47109375 0.0322265625 0.0859375
0.3486328125 0.45546875 0.05078125 0.0515625
1.044921875 0.37421875 0.0546875 0.2328125
0.71337890625 0.43828125 0.0439453125 0.1703125
0.86474609375 0.4875 0.2548828125 0.21875
-0.04296875 0.4265625 0.044921875 0.203125
0.04833984375 0

FileNotFoundError: [Errno 2] No such file or directory: '/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_train1/labels'